# Direct Generation Baseline for Omnilex Legal Retrieval

This notebook implements a **direct generation approach** where we prompt a local LLM to generate Swiss legal citations based on the query.

## Approach
1. Load a local LLM (GGUF format via llama-cpp-python)
2. For each query, prompt the LLM to directly generate relevant citations
3. Parse and normalize the generated citations
4. Create submission file

## Requirements
- llama-cpp-python (optional - runs in mock mode if not installed)
- A GGUF model file (optional - uses mock responses for demonstration if not available)
- pandas, tqdm

**Note**: This notebook runs in "mock mode" if llama-cpp-python is not installed or no model file is available, generating placeholder citations for demonstration purposes.

## 1. Setup & Configuration

In [20]:
import os
import sys
from pathlib import Path

# === CONFIGURATION ===
# Choose which dataset to run on: "val" or "test"
DATASET_MODE = "val"  # Change to "test" for final submission

# Detect environment
KAGGLE_ENV = "KAGGLE_KERNEL_RUN_TYPE" in os.environ

if KAGGLE_ENV:
    # Kaggle paths
    DATA_PATH = Path("/kaggle/input/omnilex-data")
    MODEL_PATH = Path("/kaggle/input/llama-model")
    OUTPUT_PATH = Path("/kaggle/working")
    sys.path.insert(0, "/kaggle/input/omnilex-utils")
else:
    # Local development paths
    REPO_ROOT = Path(".").resolve().parent
    DATA_PATH = REPO_ROOT / "data"
    MODEL_PATH = REPO_ROOT / "models"
    OUTPUT_PATH = REPO_ROOT / "output"
    sys.path.insert(0, str(REPO_ROOT / "src"))

# Derived paths based on DATASET_MODE
QUERY_FILE = DATA_PATH / f"{DATASET_MODE}.csv"
IS_VALIDATION_MODE = DATASET_MODE == "val"

# Create output directory
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

print(f"Environment: {'Kaggle' if KAGGLE_ENV else 'Local'}")
print(f"Dataset mode: {DATASET_MODE}")
print(f"Query file: {QUERY_FILE}")
print(f"Validation mode: {IS_VALIDATION_MODE}")
print(f"Data path: {DATA_PATH}")
print(f"Model path: {MODEL_PATH}")
print(f"Output path: {OUTPUT_PATH}")

Environment: Local
Dataset mode: val
Query file: /home/arijo/Omnilex-Agentic-Retrieval-Competition/data/val.csv
Validation mode: True
Data path: /home/arijo/Omnilex-Agentic-Retrieval-Competition/data
Model path: /home/arijo/Omnilex-Agentic-Retrieval-Competition/models
Output path: /home/arijo/Omnilex-Agentic-Retrieval-Competition/output


In [21]:
# Configuration
CONFIG = {
    # Model settings
    "model_file": "mistral-7b-instruct-v0.2.Q4_K_M.gguf",  # Adjust to your model
    "n_ctx": 4096,         # Context window
    "n_threads": 4,        # CPU threads
    "n_gpu_layers": -1,    # GPU layers (-1 = offload all layers to GPU)
    
    # Generation settings
    "max_tokens": 512,
    "temperature": 0.0,    # Low temperature for consistency
    
    # Paths
    "test_file": "test.csv",
    "train_file": "train.csv",  # For local evaluation
}

## 2. Load Local LLM

In [22]:
# Try to import llama_cpp, with fallback for environments without it
try:
    from llama_cpp import Llama
    LLAMA_AVAILABLE = True
except ImportError:
    LLAMA_AVAILABLE = False
    print("llama_cpp not available. Using mock LLM for demonstration.")

llm = None

if LLAMA_AVAILABLE:
    # Find model file
    model_file = MODEL_PATH / CONFIG["model_file"]

    if not model_file.exists():
        # Try to find any GGUF file
        gguf_files = list(MODEL_PATH.glob("*.gguf")) + list(MODEL_PATH.rglob("*.gguf"))
        if gguf_files:
            model_file = gguf_files[0]
            print(f"Using model: {model_file}")
        else:
            print(f"No model found in {MODEL_PATH}. Using mock LLM for demonstration.")
            model_file = None

    if model_file and model_file.exists():
        print(f"Loading model: {model_file}")
        llm = Llama(
            model_path=str(model_file),
            n_ctx=CONFIG["n_ctx"],
            n_threads=CONFIG["n_threads"],
            n_gpu_layers=CONFIG["n_gpu_layers"],
            verbose=False,
        )
        print("Model loaded successfully!")
        
        # Report compute device
        if CONFIG["n_gpu_layers"] == -1:
            print("Running on: GPU (all layers offloaded)")
        elif CONFIG["n_gpu_layers"] > 0:
            print(f"Running on: GPU ({CONFIG['n_gpu_layers']} layers offloaded)")
        else:
            print(f"Running on: CPU ({CONFIG['n_threads']} threads)")

if llm is None:
    print("Running in mock mode - will generate placeholder citations")

Loading model: /home/arijo/Omnilex-Agentic-Retrieval-Competition/models/mistral-7b-instruct-v0.2.Q4_K_M.gguf


llama_context: n_ctx_per_seq (4096) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


Model loaded successfully!
Running on: GPU (all layers offloaded)


## 3. Define Generation Prompt

In [23]:
SYSTEM_PROMPT = """You are a Swiss legal citation expert. Output ONLY a Python list of citations.

CITATION FORMATS:
- Federal laws: "Art. X ABBREV" where ABBREV is ZGB, OR, StGB, BV, etc.
- Court decisions: "BGE X Y Z" or "BGE X Y Z E. N" with consideration number

OUTPUT FORMAT: Python list like ["citation1", "citation2", ...]

EXAMPLES:

Query: What are the requirements for a valid contract under Swiss law?
["Art. 1 OR", "Art. 11 OR", "Art. 12 OR", "BGE 119 II 449 E. 2", "BGE 127 III 248 E. 3.1"]

Query: When can a marriage be annulled in Switzerland?
["Art. 104 ZGB", "Art. 105 ZGB", "Art. 106 ZGB", "BGE 121 III 38 E. 2b"]

Query: What constitutes negligent homicide under Swiss criminal law?
["Art. 117 StGB", "Art. 12 StGB", "BGE 116 IV 306 E. 1a"]

Query: What are the grounds for divorce in Swiss law?
["Art. 111 ZGB", "Art. 112 ZGB", "Art. 114 ZGB", "Art. 115 ZGB", "BGE 130 III 585 E. 2.1"]

Query: How is inheritance distributed under Swiss law?
["Art. 457 ZGB", "Art. 462 ZGB", "Art. 471 ZGB", "BGE 132 III 305 E. 3.2"]

Now answer:"""

# Common Swiss law abbreviations for regex matching
LAW_ABBREVS = (
    "ZGB|OR|StGB|BV|SchKG|ZPO|StPO|BGG|VwVG|IPRG|KG|DSG|MSchG|URG|PatG|"
    "DesG|UWG|PrSG|FINMAG|BankG|VAG|KAG|GwG|BEHG|FinfraG|FIDLEG|FINIG|"
    "ATSG|AHV|IV|EO|ALV|KVG|UVG|BVG|ArG|GlG|USG|RPG|WaG|JSG|TSchG|"
    "LwG|PBG|EBG|SVG|LFG|SebG|SpG|BoeB|EMRK|SR|AS|BBl|ParlG|RVOG|RVOV|"
    "MG|BPG|BPV|VBGÖ|VDSG|MWSTG|DBG|StHG|VStG|StG|ZG|CO|CP|CC|CPC|CPP|"
    "LEtr|LAsi|LN|LDIP|LCart|LDA|LPM|LBI|LDes|LCD|LFINMA|LB|LSA|LPCC|"
    "LBA|LBVM|LIMF|LSFin|LEFin|LAVS|LAI|LAPG|LACI|LAMal|LAA|LPP|LTr|"
    "LEg|LPD|LPE|LAT|LFo|LChP|LPN|LAgr|LTV|LCdF|LNA|LPTh|LTAF|LTF"
)


def extract_citations(raw_output: str) -> list[str]:
    """Extract citations from raw LLM output using regex patterns."""
    import re

    citations = []

    # Pattern for BGE citations: BGE 141 II 345 E. 3.2
    # Matches: BGE + volume + part (roman) + page + optional consideration
    bge_pattern = r'BGE\s+(\d+)\s+([IVX]+[a-z]?)\s+(\d+)(?:\s+E\.\s*([\d.a-z/]+))?'
    for match in re.finditer(bge_pattern, raw_output):
        vol, part, page, consid = match.groups()
        if consid:
            citations.append(f"BGE {vol} {part} {page} E. {consid}")
        else:
            citations.append(f"BGE {vol} {part} {page}")

    # Pattern for Art. citations with Abs./lit./Ziff.
    # Matches: Art. 221 Abs. 1 lit. b StPO, Art. 364 Abs. 1 OR, Art. 1 ZGB
    art_pattern = rf'Art\.?\s*(\d+[a-z]?)(?:\s+(Abs\.?\s*\d+))?(?:\s+(lit\.?\s*[a-z]))?(?:\s+(Ziff\.?\s*\d+))?\s+({LAW_ABBREVS})\b'
    for match in re.finditer(art_pattern, raw_output, re.IGNORECASE):
        art_num, abs_part, lit_part, ziff_part, abbrev = match.groups()
        parts = [f"Art. {art_num}"]
        if abs_part:
            # Normalize "Abs1" or "Abs 1" to "Abs. 1"
            abs_normalized = re.sub(r'Abs\.?\s*', 'Abs. ', abs_part)
            parts.append(abs_normalized.strip())
        if lit_part:
            lit_normalized = re.sub(r'lit\.?\s*', 'lit. ', lit_part)
            parts.append(lit_normalized.strip())
        if ziff_part:
            ziff_normalized = re.sub(r'Ziff\.?\s*', 'Ziff. ', ziff_part)
            parts.append(ziff_normalized.strip())
        parts.append(abbrev.upper())
        citations.append(" ".join(parts))

    # Remove duplicates while preserving order
    seen = set()
    unique = []
    for c in citations:
        if c not in seen:
            seen.add(c)
            unique.append(c)

    return unique


def generate_citations(query: str) -> list[str]:
    """Generate citations using direct LLM prompting."""
    if llm is None:
        raise RuntimeError("LLM not loaded - please ensure model is available")

    prompt = f"[INST] {SYSTEM_PROMPT}\n\nQuery: {query} [/INST]"

    response = llm(
        prompt,
        max_tokens=CONFIG["max_tokens"],
        temperature=CONFIG["temperature"],
        stop=["[INST]", "</s>", "Query:", "\n\n"],
    )

    raw_output: str = response["choices"][0]["text"].strip()  # type: ignore[index]
    print(f"Raw output: {raw_output}\n")

    # Parse Python list format first
    import re
    import ast
    
    citations = []
    
    # Try to parse as Python list
    try:
        list_match = re.search(r'\[.*?\]', raw_output, re.DOTALL)
        if list_match:
            parsed = ast.literal_eval(list_match.group())
            if isinstance(parsed, list):
                # Extract citations from parsed list items
                for item in parsed:
                    item_str = str(item).strip()
                    # Extract citations from each item (may have descriptions in parens)
                    extracted = extract_citations(item_str)
                    citations.extend(extracted)
    except (ValueError, SyntaxError):
        pass
    
    # If no citations found from list parsing, try extracting from full output
    if not citations:
        citations = extract_citations(raw_output)

    return citations

In [24]:
# Test generation with a sample query
test_query = "What are the requirements for a valid contract under Swiss law?"
print(f"Query: {test_query}")
raw_citations = generate_citations(test_query)
print("\nGenerated citations:")
for c in raw_citations:
    print(f"  - {c}")

Query: What are the requirements for a valid contract under Swiss law?
Raw output: ["Art. 1 OR", "Art. 11 OR", "Art. 12"]


Generated citations:
  - Art. 1 OR
  - Art. 11 OR


## 4. Load Test Data

In [25]:
import pandas as pd

# Load queries from the configured query file
if not QUERY_FILE.exists():
    raise FileNotFoundError(f"Query file not found: {QUERY_FILE}")

test_df = pd.read_csv(QUERY_FILE)

print(f"Loaded {len(test_df)} queries from {QUERY_FILE}")
print(f"Columns: {list(test_df.columns)}")

if IS_VALIDATION_MODE and "gold_citations" in test_df.columns:
    print(f"Gold citations available for evaluation")

test_df.head()

Loaded 10 queries from /home/arijo/Omnilex-Agentic-Retrieval-Competition/data/val.csv
Columns: ['query_id', 'query', 'gold_citations']
Gold citations available for evaluation


,query_id,query,gold_citations
0,val_001,May a court lawfully order a three‑month exten...,Art. 390 Abs. 2 StPO;Art. 393 Abs. 1 StPO;Art....
1,val_002,A claimant holding a national vocational diplo...,Art. 21 Abs. 4 ATSG;BGE 134 V 231 E. 5.1;Art. ...
2,val_003,"A. Rivera, a Peruvian national born in 1994 an...",Art. 29 Abs. 2 BV;BGE 145 I 167 E. 4.1;1B_195/...
3,val_004,"Mr. Dalton, born in 1941 and resident in a sma...",Art. 20 Abs. 2 OR;Art. 505 Abs. 1 ZGB;Art. 469...
4,val_005,"A parent, separated from their co-parent since...",Art. 100 Abs. 1 BGG;BGE 131 III 209 E. 5;BGE 1...


## 5. Generate Predictions

In [26]:
from tqdm.notebook import tqdm

# Generate predictions
predictions = []

assert test_df is not None, "test_df must be loaded before generating predictions"

for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Generating predictions"):
    query_id = row["query_id"]
    query_text = row["query"]

    # Generate citations using regex extraction (no external normalizer needed)
    raw_citations = generate_citations(query_text)

    # Use " " (single space) when no citations found, otherwise join with ";"
    predicted = ";".join(raw_citations) if raw_citations else " "

    predictions.append({
        "query_id": query_id,
        "predicted_citations": predicted,
    })

print(f"\nGenerated predictions for {len(predictions)} queries")

Generating predictions:   0%|          | 0/10 [00:00<?, ?it/s]

Raw output: ["BGE _____ _ _ _ E. _ (decision on extension of pre-trial detention under Art. 221 Abs. 1 lit. b StPO in light of the principle of proportionality)"]

Raw output: ["BGE not yet available for this specific case"]

Raw output: This query involves a specific legal case and does not directly ask about Swiss law requirements or court decisions. However, if we assume that the query is asking about the legal justification of pretrial detention under Swiss law, the following citations could be relevant:

Raw output: ["BGE 141 II 345 E. 3.2 (testamentary capacity and holographic will)", "Art. 472 ZGB (formal requirements for holographic will)", "BGE 138 III 178 E. 2 (suspicion of third-party intervention)", "BGE 132 III 305 E. 2.1 (language comprehension and vocabulary)"]

Raw output: This query involves a specific legal situation and does not directly relate to Swiss law requirements or court decisions. Therefore, it cannot be answered with a simple list of citations. Instead, I w

In [27]:
# Preview predictions
predictions_df = pd.DataFrame(predictions)
predictions_df.head(10)

,query_id,predicted_citations
0,val_001,Art. 221 Abs. 1 lit. b STPO
1,val_002,
2,val_003,
3,val_004,BGE 141 II 345 E. 3.2;Art. 472 ZGB;BGE 138 III...
4,val_005,
5,val_006,Art. 364 Abs. 1 OR;Art. 248 Abs. 1 OR
6,val_007,Art. 451 ZGB;Art. 452 ZGB;Art. 453 ZGB;Art. 45...
7,val_008,
8,val_009,
9,val_010,Art. 12 STGB;Art. 13 STGB;Art. 114 ZGB


## 6. Create Submission

In [28]:
# Save submission
submission_path = OUTPUT_PATH / "submission.csv"
predictions_df.to_csv(submission_path, index=False)

print(f"Submission saved to: {submission_path}")
print(f"Total predictions: {len(predictions_df)}")

# Show sample
print("\nSample submission:")
print(predictions_df.head())

Submission saved to: /home/arijo/Omnilex-Agentic-Retrieval-Competition/output/submission.csv
Total predictions: 10

Sample submission:
  query_id                                predicted_citations
0  val_001                        Art. 221 Abs. 1 lit. b STPO
1  val_002                                                   
2  val_003                                                   
3  val_004  BGE 141 II 345 E. 3.2;Art. 472 ZGB;BGE 138 III...
4  val_005                                                   


## Summary

This baseline notebook demonstrates a simple direct generation approach:

1. **Prompt engineering**: We use a structured prompt that asks the LLM to generate Swiss legal citations in standard format.

2. **Citation normalization**: The generated citations are normalized to canonical form for consistent evaluation.

3. **Limitations**:
   - The LLM may hallucinate non-existent citations
   - No access to actual legal documents for verification
   - Relies entirely on the LLM's training data knowledge

For better results, see the **Agentic Retrieval Baseline** notebook which uses search tools to ground the generation in actual legal documents.